# Setup Google Drive

Mount google drive and make it the actual directory (unnecessary for local projects)

In [21]:
# import os
# cwd= os.getcwd()
# print (cwd)

In [22]:
# from google.colab import drive
# drive.mount('/drive')
# import os
# os.chdir('/drive/MyDrive/TI6')
# cwd= os.getcwd()
# print (cwd)

# Setup Yolo

Clone GitHub [repository](https://github.com/ultralytics/yolov5), install [dependencies](https://github.com/ultralytics/yolov5/blob/master/requirements.txt) and check PyTorch and GPU.

In [23]:
import os

if(not os.path.exists('yolov5') and not os.getcwd().split('/')[-1] == 'yolov5'):
  !git clone https://github.com/ultralytics/yolov5 # clone

%cd yolov5
# %pip install -qr requirements.txt comet_ml 
    
# %pip install torch torchvision torchaudio
# %pip install pyyaml

import torch
import utils
# display = utils.notebook_init()  # checks

import os
cwd= os.getcwd()
print (cwd)

[Errno 2] No such file or directory: 'yolov5'
/home/luis/Documents/TI6/yolov5
/home/luis/Documents/TI6/yolov5


# 1. Train

Train a YOLOv5s model on the [CSDD](https://github.com/Kerio99/CSDD) dataset with `--data coco128.yaml`, starting from pretrained `--weights yolov5s.pt`.

- **Training Results** are saved to `runs/train/` with incrementing run directories.

In [24]:
#@title Select YOLOv5 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'ClearML', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'ClearML':
  %pip install -q clearml
  import clearml; clearml.browser_login()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir runs/train

Note: you may need to restart the kernel to use updated packages.


COMET WARNING: comet_ml.init() is deprecated and will be removed soon. Please use comet_ml.login()


## Train YOLOv5

In [25]:
image_size = 1536 #@param {type:"integer"}
batch_size = 3 #@param {type:"integer"}
# 8 funciona no completo com 2048px
epochs = 100 #@param {type:"integer"}
yaml = 'CSDD.yaml' #@param {type:"string"}
remove_previous_tests = False #@param {type:"boolean"}
train_from_scratch = False #@param {type:"boolean"}

In [26]:
# Remover diretórios de execuções anteriores de treinamento e validação
if remove_previous_tests:
  import shutil
  for folder in ['runs/train', 'runs/val', 'runs/detect']:
      if os.path.exists(folder):
          shutil.rmtree(folder)

In [27]:
# Train YOLOv5s on CSDD
if train_from_scratch:
    !python train.py --img {image_size} --batch {batch_size} --epochs {epochs} --data {yaml} --weights yolov5n.pt --cache

In [28]:
import os
import glob

# Find the latest run directory
list_of_files = glob.glob('runs/train/exp*') # Use glob to find directories starting with 'exp'
if list_of_files:
    latest_file = max(list_of_files, key=os.path.getctime)

    # Construct the base new directory name
    base_new_dir_name = f'runs/train/{yaml.split(".")[0]}B{batch_size}_E{epochs}_I{image_size}'
    new_dir_name = base_new_dir_name
    counter = 1

    # Check if the directory name already exists and append a counter if it does
    while os.path.exists(new_dir_name):
        new_dir_name = f'{base_new_dir_name}-{counter}'
        counter += 1

    # Rename the directory
    os.rename(latest_file, new_dir_name)
    print(f"Renomeado '{latest_file}' para '{new_dir_name}'")
else:
    print("Nenhuma pasta de treinamento 'exp' encontrada para renomear.")

Nenhuma pasta de treinamento 'exp' encontrada para renomear.


# 2. Detect (test)
Validate a model's accuracy on the [COCO](https://cocodataset.org/#home) dataset's `val` or `test` splits. Models are downloaded automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases). To show results by class use the `--verbose` flag.

Este código realiza os seguintes passos:

1.  **Lista as pastas de treinamento:** Verifica o diretório `runs/train/` e lista todas as subpastas encontradas. Cada subpasta representa um treinamento realizado.
2.  **Pede para o usuário escolher:** Imprime as pastas disponíveis com um número correspondente e solicita que o usuário digite o número da pasta que contém os pesos do modelo que ele deseja usar para a detecção.
3.  **Define o caminho dos pesos:** Com base na escolha do usuário, constrói o caminho completo para o arquivo `best.pt` dentro da pasta de treinamento selecionada e armazena esse caminho na variável `selected_weights_path`.
4.  **Executa a detecção:** Se um caminho de peso foi selecionado com sucesso, ele executa o script `detect.py` usando os pesos especificados (`--weights {selected_weights_path}`), define o tamanho da imagem para 640 (`--img 640`), um limiar de confiança de 0.25 (`--conf 0.25`) e especifica que as imagens para detecção estão no diretório `data/images` (`--source data/images`). Se nenhum peso foi selecionado (por exemplo, se não havia pastas de treinamento), ele imprime uma mensagem informando ao usuário para executar a célula anterior.

In [29]:
test_model = True #@param {type:"boolean"}

In [30]:
train_runs_path = 'runs/train/'
selected_weights_path = None # Initialize variable to store the selected path

if test_model:
    if os.path.exists(train_runs_path):
      run_dirs = [d for d in os.listdir(train_runs_path) if os.path.isdir(os.path.join(train_runs_path, d))]
      if run_dirs:
        print("Pastas de treinamento disponíveis:")
        for i, run_dir in enumerate(run_dirs):
          print(f"{i}: {run_dir}")
    
        while True:
          try:
            choice = int(input("Digite o número da pasta de treinamento que deseja usar: "))
            if 0 <= choice < len(run_dirs):
              selected_run_dir = run_dirs[choice]
              selected_weights_path = os.path.join(train_runs_path, selected_run_dir, 'weights', 'best.pt')
              print(f"Você selecionou a pasta: {selected_run_dir}")
              print(f"Caminho para os pesos selecionados: {selected_weights_path}")
              break
            else:
              print("Escolha inválida. Por favor, digite um número da lista.")
          except ValueError:
            print("Entrada inválida. Por favor, digite um número.")
      else:
        print(f"Nenhuma pasta de treinamento encontrada em {train_runs_path}. Certifique-se de ter executado o treinamento.")
    else:
      print(f"A pasta {train_runs_path} não foi encontrada. Certifique-se de ter executado o treinamento.")

Pastas de treinamento disponíveis:
0: CSDD_B3_E300_I1536
Você selecionou a pasta: CSDD_B3_E300_I1536
Caminho para os pesos selecionados: runs/train/CSDD_B3_E300_I1536/weights/best.pt


In [31]:
import Time
if selected_weights_path:
  time = Time.now()
  !python detect.py --weights {selected_weights_path} --img {image_size} --conf 0.25 --source ../datasets/CSDD_det/images/test2017/
  time = Time.now() - time
  print(f"Detecção sequencial concluída em {time} segundos.")
else:

  print("Nenhum caminho de peso selecionado. Por favor, execute a célula acima para selecionar um peso.")

ModuleNotFoundError: No module named 'Time'

In [ ]:
import time

time.time()

1760654599.137604

In [36]:
start_time = time.time()
!python detectBatchv3.py --weights {selected_weights_path} --imgsz {image_size} --conf 0.25 --source ../datasets/CSDD_det/images/test2017/ --workers 0 --batch-size 24
end_time = time.time() - start_time

detectBatchv3: weights=['runs/train/CSDD_B3_E300_I1536/weights/best.pt'], source=../datasets/CSDD_det/images/test2017/, data=data/coco128.yaml, imgsz=[1536, 1536], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1, batch_size=24, workers=0
YOLOv5 🚀 v7.0-433-ga1019635 Python-3.12.3 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1650, 3897MiB)

Fusing layers... 
Model summary: 157 layers, 1763224 parameters, 0 gradients, 4.1 GFLOPs
Using 0 workers for background pre-processing
Image batching is ON, Workers: 0 (not using background pre-processing)
⚠️  WARNING: NÚMERO DE WORKERS = 0, USANDO MODO BATCHING SEM PRÉ-PROCESSAMENTO ASSÍNCRONO
0007.jpg: 7 scratchs, 2 spo

In [ ]:
#Teste de detecção sequencial e em lote
import csv
import time

csvfile = './times.csv'
if(os.path.exists(csvfile)):
    os.remove(csvfile)

with open('./times.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['workers', 'batch_size', 'tempo(s)'])
    
    start_time = time.time()
    !python detect.py --weights {selected_weights_path} --imgsz {image_size} --conf 0.25 --source ../datasets/CSDD_det/images/test2017/
    end_time = time.time() - start_time
    writer.writerow([0, 0, end_time])
    print(f"Detecção sequencial concluída em {end_time} segundos.")

    for workers in [0, 1, 2, 3, 4, 5, 6]:
        for batch_size in [1, 2, 4, 8, 16, 24]:
            start_time = time.time()
            !python detectBatchv3.py --weights {selected_weights_path} --imgsz {image_size} --conf 0.25 --source ../datasets/CSDD_det/images/test2017/ --workers {workers} --batch-size 32
            end_time = time.time() - start_time
            writer.writerow([workers, batch_size, end_time])
            print(f"Detecção em lote concluída em {end_time} segundos.")

#!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images
#!python val.py --weights yolov5s.pt --data coco.yaml --img 640 --half

detect: weights=['runs/train/CSDD_B3_E300_I1536/weights/best.pt'], source=../datasets/CSDD_det/images/test2017/, data=data/coco128.yaml, imgsz=[1536, 1536], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-433-ga1019635 Python-3.12.3 torch-2.8.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1650, 3897MiB)

Fusing layers... 
Model summary: 157 layers, 1763224 parameters, 0 gradients, 4.1 GFLOPs
image 1/422 /home/luis/Documents/TI6/datasets/CSDD_det/images/test2017/0007.jpg: 1536x1536 7 scratchs, 2 spots, 30.3ms
image 2/422 /home/luis/Documents/TI6/datasets/CSDD_det/images/test2017/0010.jpg: 1536x1536 7 scratchs, 2 spots, 29.9ms
image 3/422 /home/luis/Docume